# Sistema de recomendación por filtrado de contenido
Las recomendaciones están basadas en las características de cada ítem

## Bibliotecas

In [2]:
import numpy as np
import pandas as pd

## Lectura del anime dataset de kaggle

In [3]:
anime_titles = pd.read_csv('anime_cleaned.csv') #Cargamos el archivo donde esta contenido el dataset de los animes

In [4]:
anime_titles.head() #mostrar los 5 primeros titulos del dataset

,anime_id,title,title_english,title_japanese,title_synonyms,image_url,type,source,episodes,status,...,premiered,broadcast,related,producer,licensor,studio,genre,opening_theme,ending_theme,duration_min
0,11013,Inu x Boku SS,Inu X Boku Secret Service,妖狐×僕SS,Youko x Boku SS,https://myanimelist.cdn-dena.com/images/anime/...,TV,Manga,12,Finished Airing,...,Winter 2012,Fridays at Unknown,"{'Adaptation': [{'mal_id': 17207, 'type': 'man...","Aniplex, Square Enix, Mainichi Broadcasting Sy...",Sentai Filmworks,David Production,"Comedy, Supernatural, Romance, Shounen","['""Nirvana"" by MUCC']","['#1: ""Nirvana"" by MUCC (eps 1, 11-12)', '#2: ...",24.0
1,2104,Seto no Hanayome,My Bride is a Mermaid,瀬戸の花嫁,The Inland Sea Bride,https://myanimelist.cdn-dena.com/images/anime/...,TV,Manga,26,Finished Airing,...,Spring 2007,Unknown,"{'Adaptation': [{'mal_id': 759, 'type': 'manga...","TV Tokyo, AIC, Square Enix, Sotsu",Funimation,Gonzo,"Comedy, Parody, Romance, School, Shounen","['""Romantic summer"" by SUN&LUNAR']","['#1: ""Ashita e no Hikari (明日への光)"" by Asuka Hi...",24.0
2,5262,Shugo Chara!! Doki,Shugo Chara!! Doki,しゅごキャラ！！どきっ,"Shugo Chara Ninenme, Shugo Chara! Second Year",https://myanimelist.cdn-dena.com/images/anime/...,TV,Manga,51,Finished Airing,...,Fall 2008,Unknown,"{'Adaptation': [{'mal_id': 101, 'type': 'manga...","TV Tokyo, Sotsu",NaN,Satelight,"Comedy, Magic, School, Shoujo","['#1: ""Minna no Tamago (みんなのたまご)"" by Shugo Cha...","['#1: ""Rottara Rottara (ロッタラ ロッタラ)"" by Buono! ...",24.0
3,721,Princess Tutu,Princess Tutu,プリンセスチュチュ,NaN,https://myanimelist.cdn-dena.com/images/anime/...,TV,Original,38,Finished Airing,...,Summer 2002,Fridays at Unknown,"{'Adaptation': [{'mal_id': 1581, 'type': 'mang...","Memory-Tech, GANSIS, Marvelous AQL",ADV Films,Hal Film Maker,"Comedy, Drama, Magic, Romance, Fantasy","['""Morning Grace"" by Ritsuko Okazaki']","['""Watashi No Ai Wa Chiisaikeredo"" by Ritsuko ...",16.0
4,12365,Bakuman. 3rd Season,Bakuman.,バクマン。,Bakuman Season 3,https://myanimelist.cdn-dena.com/images/anime/...,TV,Manga,25,Finished Airing,...,Fall 2012,Unknown,"{'Adaptation': [{'mal_id': 9711, 'type': 'mang...","NHK, Shueisha",NaN,J.C.Staff,"Comedy, Drama, Romance, Shounen","['#1: ""Moshimo no Hanashi (もしもの話)"" by nano.RIP...","['#1: ""Pride on Everyday"" by Sphere (eps 1-13)...",24.0


In [5]:
#Usar solo las columnas de anime_id, title, genre
anime_df = pd.read_csv('anime_cleaned.csv', usecols=[0,1,28])
anime_df.head()

,anime_id,title,genre
0,11013,Inu x Boku SS,"Comedy, Supernatural, Romance, Shounen"
1,2104,Seto no Hanayome,"Comedy, Parody, Romance, School, Shounen"
2,5262,Shugo Chara!! Doki,"Comedy, Magic, School, Shoujo"
3,721,Princess Tutu,"Comedy, Drama, Magic, Romance, Fantasy"
4,12365,Bakuman. 3rd Season,"Comedy, Drama, Romance, Shounen"


## Variables dummy: géneros

In [6]:
# Convertimos los generos de anime a variables dummy y los concatenamos a la tabla como columnas
anime_df = pd.concat([anime_df, anime_df.genre.str.get_dummies(sep=', ')], axis=1) #Separamos los generos por ', '
anime_df.head() #Para cada anime, las columnas tendrán un valor de 0 excepto para aquellos géneros que tenga el anime

,anime_id,title,genre,Action,Adventure,Cars,Comedy,Dementia,Demons,Drama,...,Shounen Ai,Slice of Life,Space,Sports,Super Power,Supernatural,Thriller,Vampire,Yaoi,Yuri
0,11013,Inu x Boku SS,"Comedy, Supernatural, Romance, Shounen",0,0,0,1,0,0,0,...,0,0,0,0,0,1,0,0,0,0
1,2104,Seto no Hanayome,"Comedy, Parody, Romance, School, Shounen",0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,5262,Shugo Chara!! Doki,"Comedy, Magic, School, Shoujo",0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,721,Princess Tutu,"Comedy, Drama, Magic, Romance, Fantasy",0,0,0,1,0,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12365,Bakuman. 3rd Season,"Comedy, Drama, Romance, Shounen",0,0,0,1,0,0,1,...,0,0,0,0,0,0,0,0,0,0


## Creación de la lista de index "anime_categories"

In [7]:
anime_categories = anime_df.columns[3:] #tabla de categorias de anime (empiezan desde el index 3 de anime_df)
anime_df.loc[0] #imprimir caracteristicas del primer titulo en row=0

anime_id                                          11013
title                                     Inu x Boku SS
genre            Comedy, Supernatural, Romance, Shounen
Action                                                0
Adventure                                             0
Cars                                                  0
Comedy                                                1
Dementia                                              0
Demons                                                0
Drama                                                 0
Ecchi                                                 0
Fantasy                                               0
Game                                                  0
Harem                                                 0
Hentai                                                0
Historical                                            0
Horror                                                0
Josei                                           

## Ordered Dict para crear las preferencias de usuario
### Usuario 1: Gustos enfocados en animes de acción, ciencia ficción, policiaco y misterio

In [8]:
from collections import OrderedDict #Creamos un diccionario ordenado porque nos interesa mantener el orden de los generos

user_preferences = OrderedDict(zip(anime_categories, [])) #Crear un usuario con preferencias en acción, ciencia ficcion y misterio

user_preferences['Action'] = 5
user_preferences['Adventure'] = 1
user_preferences['Cars'] = 1
user_preferences['Comedy'] = 1
user_preferences['Dementia'] = 2
user_preferences['Demons'] = 1
user_preferences['Drama'] = 3
user_preferences['Ecchi'] = 1
user_preferences['Fantasy'] = 3
user_preferences['Game'] = 5
user_preferences['Harem'] = 1
user_preferences['Hentai'] = 1
user_preferences['Historical'] = 2
user_preferences['Horror'] = 5
user_preferences['Josei'] = 5
user_preferences['Kids'] = 1
user_preferences['Magic'] = 1
user_preferences['Martial Arts'] =1
user_preferences['Mecha'] = 5
user_preferences['Military'] = 3
user_preferences['Music'] = 1
user_preferences['Mystery'] = 5
user_preferences['Parody'] = 1
user_preferences['Police'] = 5
user_preferences['Psychological'] = 5
user_preferences['Romance'] = 1
user_preferences['Samuarai'] = 1
user_preferences['School'] = 1
user_preferences['Sci-Fi'] = 5
user_preferences['Seinen'] = 5
user_preferences['Shoujo'] = 1
user_preferences['Shoujo Ai'] = 1
user_preferences['Shounen'] = 1
user_preferences['Shounen Ai'] = 2
user_preferences['Slice of Life'] = 2
user_preferences['Space'] = 5
user_preferences['Sports'] = 1
user_preferences['Super Power'] = 2
user_preferences['Supernatural'] = 3
user_preferences['Thriller'] = 5
user_preferences['Vampire'] = 1
user_preferences['Yaoi'] = 1
user_preferences['Yuri'] = 1

### Usuario 2: Gustos enfocados en animes de romance, fantasía, música, shoujo y vampiros

In [9]:
user2_preferences = OrderedDict(zip(anime_categories, [])) #Crear un usuario con preferencias femeninas

user2_preferences['Action'] = 1
user2_preferences['Adventure'] = 3
user2_preferences['Cars'] = 1
user2_preferences['Comedy'] = 3
user2_preferences['Dementia'] = 2
user2_preferences['Demons'] = 4
user2_preferences['Drama'] = 5
user2_preferences['Ecchi'] = 1
user2_preferences['Fantasy'] = 5
user2_preferences['Game'] = 2
user2_preferences['Harem'] = 1
user2_preferences['Hentai'] = 1
user2_preferences['Historical'] = 1
user2_preferences['Horror'] = 1
user2_preferences['Josei'] = 1
user2_preferences['Kids'] = 3
user2_preferences['Magic'] = 5
user2_preferences['Martial Arts'] =1
user2_preferences['Mecha'] = 1
user2_preferences['Military'] = 1
user2_preferences['Music'] = 5
user2_preferences['Mystery'] = 3
user2_preferences['Parody'] = 1
user2_preferences['Police'] = 1
user2_preferences['Psychological'] = 1
user2_preferences['Romance'] = 5
user2_preferences['Samuarai'] = 1
user2_preferences['School'] = 5
user2_preferences['Sci-Fi'] = 1
user2_preferences['Seinen'] = 1
user2_preferences['Shoujo'] = 5
user2_preferences['Shoujo Ai'] = 5
user2_preferences['Shounen'] = 1
user2_preferences['Shounen Ai'] = 3
user2_preferences['Slice of Life'] = 4
user2_preferences['Space'] = 1
user2_preferences['Sports'] = 1
user2_preferences['Super Power'] = 2
user2_preferences['Supernatural'] = 3
user2_preferences['Thriller'] = 5
user2_preferences['Vampire'] = 5
user2_preferences['Yaoi'] = 1
user2_preferences['Yuri'] = 1

### Usuario 3: Gustos enfocados en harem

In [23]:
user3_preferences = OrderedDict(zip(anime_categories, [])) #Crear un usuario con preferencias en ecchi

user3_preferences['Action'] = 3
user3_preferences['Adventure'] = 3
user3_preferences['Cars'] = 1
user3_preferences['Comedy'] = 5
user3_preferences['Dementia'] = 1
user3_preferences['Demons'] = 5
user3_preferences['Drama'] = 2
user3_preferences['Ecchi'] = 5
user3_preferences['Fantasy'] = 1
user3_preferences['Game'] = 5
user3_preferences['Harem'] = 5
user3_preferences['Hentai'] = 5
user3_preferences['Historical'] = 1
user3_preferences['Horror'] = 1
user3_preferences['Josei'] = 1
user3_preferences['Kids'] = 1
user3_preferences['Magic'] = 2
user3_preferences['Martial Arts'] =3
user3_preferences['Mecha'] = 1
user3_preferences['Military'] = 4
user3_preferences['Music'] = 1
user3_preferences['Mystery'] = 1
user3_preferences['Parody'] = 5
user3_preferences['Police'] = 1
user3_preferences['Psychological'] = 1
user3_preferences['Romance'] = 3
user3_preferences['Samuarai'] = 1
user3_preferences['School'] = 3
user3_preferences['Sci-Fi'] = 1
user3_preferences['Seinen'] = 1
user3_preferences['Shoujo'] = 1
user3_preferences['Shoujo Ai'] = 4
user3_preferences['Shounen'] = 1
user3_preferences['Shounen Ai'] = 4
user3_preferences['Slice of Life'] = 4
user3_preferences['Space'] = 1
user3_preferences['Sports'] = 1
user3_preferences['Super Power'] = 1
user3_preferences['Supernatural'] = 1
user3_preferences['Thriller'] = 1
user3_preferences['Vampire'] = 5
user3_preferences['Yaoi'] = 5
user3_preferences['Yuri'] = 5

## Función: Producto punto de la matriz usuario por la matriz de variables dummy

In [24]:
#Calcular las puntuaciones de este usuario hipotetico (se podria usar np.dot pero lo hice a mano)
def dot_product(vector_1, vector_2):
    """Funcion que realiza el producto punto entre dos matrices
        ARGS: 
            vector_1: una matriz de nxm
            vector_2: una matriz de nxm
        RETURNS:
            La suma del producto punto de todos los elementos"""
    return sum([ i*j for i,j in zip(vector_1, vector_2)])

## Testear a los 3 usuarios con el primer título del dataset

In [25]:
Inu_x_Boku_SS_features = anime_df.loc[0][anime_categories] #Imprimir una tabla de géneros para el primer título
Inu_x_Boku_SS_features

Action           0
Adventure        0
Cars             0
Comedy           1
Dementia         0
Demons           0
Drama            0
Ecchi            0
Fantasy          0
Game             0
Harem            0
Hentai           0
Historical       0
Horror           0
Josei            0
Kids             0
Magic            0
Martial Arts     0
Mecha            0
Military         0
Music            0
Mystery          0
Parody           0
Police           0
Psychological    0
Romance          1
Samurai          0
School           0
Sci-Fi           0
Seinen           0
Shoujo           0
Shoujo Ai        0
Shounen          1
Shounen Ai       0
Slice of Life    0
Space            0
Sports           0
Super Power      0
Supernatural     1
Thriller         0
Vampire          0
Yaoi             0
Yuri             0
Name: 0, dtype: object

In [26]:
Inu_x_Boku_SS_predicted_score = dot_product(Inu_x_Boku_SS_features, user_preferences.values()) 
print("User 1:",Inu_x_Boku_SS_predicted_score) #Predecimos la puntuación del usuario 1

Inu_x_Boku_SS_predicted_score = dot_product(Inu_x_Boku_SS_features, user2_preferences.values()) 
print("User 2:",Inu_x_Boku_SS_predicted_score) #Predecimos la puntuación del usuario 2

Inu_x_Boku_SS_predicted_score = dot_product(Inu_x_Boku_SS_features, user3_preferences.values()) 
print("User 3:",Inu_x_Boku_SS_predicted_score) #Predecimos la puntuación del usuario 3

User 1: 6
User 2: 12
User 3: 10


## Testear ahora con un título policiaco

In [27]:
anime_df[anime_df.title.str.contains('Psycho-Pass')] #Ahora predeciremos la calificacion del usuario para una serie policiaca

,anime_id,title,genre,Action,Adventure,Cars,Comedy,Dementia,Demons,Drama,...,Slice of Life,Space,Sports,Super Power,Supernatural,Thriller,Vampire,Yaoi,Yuri,score
3226,13601,Psycho-Pass,"Action, Police, Psychological, Sci-Fi",1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,7
3935,21339,Psycho-Pass Movie,"Action, Military, Police, Sci-Fi",1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,7
4152,37442,Psycho-Pass SS Case 3: Onshuu no Kanata ni,"Action, Police, Psychological, Sci-Fi",1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,7
8658,23281,Psycho-Pass 2,"Action, Sci-Fi, Police, Psychological",1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,7
9104,37440,Psycho-Pass SS Case 1: Tsumi to Batsu,"Action, Police, Psychological, Sci-Fi",1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,7
13528,37441,Psycho-Pass SS Case 2: First Guardian,"Action, Police, Psychological, Sci-Fi",1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,7


In [28]:
psycho_pass_id = 13601 #De la tabla anterior, localizamos el id del título de la columna anime_id
psycho_pass_features = anime_df[anime_df.anime_id==psycho_pass_id][anime_categories] #Obtener lista de géneros del título
psycho_pass_features.T #Transpuesta para que se imprima y se lea mas comodo

,3226
Action,1
Adventure,0
Cars,0
Comedy,0
Dementia,0
Demons,0
Drama,0
Ecchi,0
Fantasy,0
Game,0


In [29]:
psycho_pass_user_predicted_score = dot_product(psycho_pass_features.values[0], user_preferences.values())
print("User 1:",psycho_pass_user_predicted_score) #Calculamos el score de Psycho-Pass para user 1 

psycho_pass_user_predicted_score = dot_product(psycho_pass_features.values[0], user2_preferences.values())
print("User 2:",psycho_pass_user_predicted_score) #Calculamos el score de Psycho-Pass para user 1 

psycho_pass_user_predicted_score = dot_product(psycho_pass_features.values[0], user3_preferences.values())
print("User 3:",psycho_pass_user_predicted_score) #Calculamos el score de Psycho-Pass para user 1 

User 1: 20
User 2: 4
User 3: 6


## Funcion que genera las 10 mejores recomendaciones para cada usuario

In [30]:
def get_anime_recommendations(user_preferences, n_recommendations):
    #metemos una columna al dataset anime_df con la puntuacion calculada para el usuario
    anime_df['score'] = anime_df[anime_categories].apply(dot_product, 
                                                           args=([user_preferences.values()]), axis=1)
    return anime_df.sort_values(by=['score'], ascending=False)['title'][:n_recommendations]

print("User 1 top 10:\n")
print(get_anime_recommendations(user_preferences, 10))
print("\n")
print("User 2 top 10:\n")
print(get_anime_recommendations(user2_preferences, 10))
print("\n")
print("User 3 top 10:\n")
print(get_anime_recommendations(user3_preferences, 10))

User 1 top 10:

2490     Ghost in the Shell: Stand Alone Complex - The ...
3772                                             Betterman
13232                             Nihon Animator Mihonichi
5415                                            Ani*Kuri15
723        Ghost in the Shell: Stand Alone Complex 2nd GIG
4700                                             AD Police
11802                                 Boukyaku no Senritsu
7624                                               Monster
10655            Mobile Suit Gundam Wing: Operation Meteor
1296                                               Gantz:O
Name: title, dtype: object


User 2 top 10:

5415                        Ani*Kuri15
546      Wagamama☆Fairy Mirumo de Pon!
11848           Kamikaze Kaitou Jeanne
46                   Cardcaptor Sakura
8892                   Asagiri no Miko
10230                         Mai-HiME
13519             Kagee Mukashibanashi
346                  Kagee Grimm Douwa
13879           Kagee Aesop Monogatar

## References
1. Pydata.org (2019). Pandas documentation. From: https://pandas.pydata.org/pandas-docs/stable/index.html
2. Iñigo, D. (2018). Primeros pasos en sistemas de recomendación. From: https://www.analiticaweb.es/primeros-pasos-sistemas-recomendacion/
3. Garrido, M. (2015). Breve introducción a los Sistemas de Recomendación. Pybonacci.org. From: https://www.pybonacci.org/2015/12/11/breve-introduccion-a-los-sistemas-de-recomendacion/
